<a href="https://colab.research.google.com/github/sp7412/colab/blob/master/distilling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [133]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

def softmax_sparse_categorical_crossentropy(labels, logits):
  softmaxed = tf.keras.backend.softmax(logits)

  return tf.keras.losses.sparse_categorical_crossentropy(labels, softmaxed)

In [134]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [135]:
ds_info.features['image'].shape

(28, 28, 1)

In [136]:
num_training_examples = ds_info.splits['train'].num_examples
num_training_examples

60000

In [137]:
ds_train_iter = ds_train.as_numpy_iterator()
train_images, train_labels = zip(*[ds_train_iter.next() for i in range(num_training_examples)])
train_images = np.asarray(train_images)

In [139]:
train_images.shape

(60000, 28, 28, 1)

In [140]:
num_testing_examples = ds_info.splits['test'].num_examples
num_testing_examples

10000

In [141]:
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [142]:
ds_test_iter = ds_test.as_numpy_iterator()

In [143]:
test_images, test_labels = zip(*[ds_test_iter.next() for i in range(num_testing_examples)])

In [144]:
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [176]:
control_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  tf.keras.layers.Dense(10, activation='softmax')])
  #tf.keras.layers.Dense(10, activation=None)])

control_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
#control_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
control_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_9 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_20 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_20 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_21 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_21 (Dropout)         (None, 6, 6, 64)         

In [177]:
control_model.fit(ds_train, epochs=5)

test_loss, test_acc = control_model.evaluate(ds_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 0.9724 - accuracy: 0.7401
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.3849 - accuracy: 0.9036
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.2712 - accuracy: 0.9254
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.2248 - accuracy: 0.9356
Epoch 5/5
79/79 [==============================] - 0s 2ms/step - loss: 0.1810 - accuracy: 0.9469
Test Loss: 0.18099378049373627
Test Accuracy: 0.9469000101089478


In [151]:
test_loss, test_acc = control_model.evaluate(np.asarray(test_images,dtype=np.float64),
                                             np.asarray(test_labels,dtype=np.float64))
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.1072 - accuracy: 0.9686
Test Loss: 0.10718918591737747
Test Accuracy: 0.9685999751091003


In [152]:
teacher_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

#teacher_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
teacher_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
teacher_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_13 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_13 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_14 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_14 (Dropout)         (None, 6, 6, 64)         

In [154]:
teacher_model.fit(ds_train, epochs=5)

test_loss, test_acc = teacher_model.evaluate(ds_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0574 - accuracy: 0.9826
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0514 - accuracy: 0.9838
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0477 - accuracy: 0.9851
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0456 - accuracy: 0.9863
Epoch 5/5
79/79 [==============================] - 0s 2ms/step - loss: 0.0335 - accuracy: 0.9892
Test Loss: 0.03350392356514931
Test Accuracy: 0.9891999959945679


In [155]:
ds_train_images = tf.data.Dataset.from_tensor_slices(list(map(lambda x:tf.reshape(x,(-1,28,28,1)), train_images)))
soft_labels = teacher_model.predict(ds_train_images,verbose=1)

60000/60000 [==============================] - 71s 1ms/step


In [156]:
temperature = 3
afterwards_temperature = 1

def temperature_softmax(logits):
  soft_logits = tf.keras.backend.exp(logits / temperature)
  return soft_logits / tf.keras.backend.sum(soft_logits, axis=-1, keepdims=True) / afterwards_temperature

def distillation_loss(labels, logits):
  labels = temperature_softmax(labels)
  logits = temperature_softmax(logits)

  return -tf.keras.backend.mean(labels * tf.keras.backend.log(logits))

In [159]:
student_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  # tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

student_model.compile('adam', distillation_loss, ['accuracy'])
student_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_8 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_18 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_18 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_19 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_19 (Dropout)         (None, 6, 6, 64)         

In [160]:
student_model.fit(train_images, soft_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1221 - accuracy: 0.7632
Epoch 2/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0850 - accuracy: 0.9169
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0816 - accuracy: 0.9317
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0804 - accuracy: 0.9385
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0795 - accuracy: 0.9418
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0791 - accuracy: 0.9437
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0787 - accuracy: 0.9461
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0783 - accuracy: 0.9477
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0781 - accuracy: 0.9490
Epoch 10/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.077

In [181]:
num_classes = 10
output_layer = tf.keras.layers.Dense(num_classes, activation='softmax', name='scores')(student_model.layers[-2].output)
new_model = tf.keras.Model(student_model.input, output_layer)

In [182]:
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [183]:
new_model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape_8 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_18 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_18 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_19 (Batc (None, 6, 6, 64)         

In [184]:
test_loss, test_acc = new_model.evaluate(ds_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

79/79 [==============================] - 0s 2ms/step - loss: 2.6552 - accuracy: 0.0453
Test Loss: 2.655200481414795
Test Accuracy: 0.04529999941587448


In [186]:
# test_loss, test_acc = new_model.evaluate(test_images, test_labels,batch_size=128)

79/79 [==============================] - 0s 3ms/step - loss: 2.6552 - accuracy: 0.0453
